In [3]:
import wptools, wikipedia, re

def checkAllAlpha(test_str):
    for c in test_str:
        if not c.isalpha():
            return False
    return True

def removeBracketsData(test_str):
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == '{':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif i == '}'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret
def populate_data(company_name):
    try:
        company_data_dict = wptools.page(wikipedia.search(company_name)[0]).get_parse().data['infobox']
        comp_data_to_store = {'name': company_name, 'type': '', 'industry': set(), 'revenue': -1, 'hq_location': '', 'num_employees': 0}

        if 'name' in company_data_dict.keys():
            comp_data_to_store['name'] = company_data_dict['name']

        if 'type' in company_data_dict.keys():
            str_to_process = company_data_dict['type']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                if len(token)>0:
                    comp_data_to_store['type'] = token.split('|')[-1]
                    break

        if 'industry' in company_data_dict.keys():
            str_to_process = company_data_dict['industry']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and checkAllAlpha(token[0]):
                    for tok in token.split('|'):
                        if len(tok)>0:
                            comp_data_to_store['industry'].add(tok)

        if 'revenue' in company_data_dict.keys():
            str_to_process = company_data_dict['revenue']
            index = -1
            revenue = ''
            if 'million' in str_to_process:
                index = str_to_process.index('million')
                revenue = 'million'
            elif 'billion' in str_to_process:
                index = str_to_process.index('billion')
                revenue = 'billion'
            elif 'thousand' in str_to_process:
                index = str_to_process.index('thousand')
                revenue = 'thousand'
            if revenue!='':
                l = index-1
                while str_to_process[l].isdigit() or str_to_process[l]==' 'or str_to_process[l]=='.':
                    revenue = str_to_process[l] + revenue
                    l-=1
                comp_data_to_store['revenue'] = revenue.strip()

        if 'hq_location' in company_data_dict.keys():
            str_to_process = company_data_dict['hq_location']
            list_of_tokens = re.findall(r'([^\[\]]*)', str_to_process)
            for token in list_of_tokens:
                token = token.strip()
                if len(token)>0 and token[0].isalpha():
                    for tok in token.split('|'):
                        if len(tok)>0:
                            comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'] + tok + ' '
                    comp_data_to_store['hq_location'] = comp_data_to_store['hq_location'].strip()

        if 'num_employees' in company_data_dict.keys():
            str_to_process = removeBracketsData(company_data_dict['num_employees'])
            n_emp = 0
            skip = False
            for c in str_to_process:

                if c.isdigit():
                    n_emp = n_emp*10 + int(c)
            comp_data_to_store['num_employees'] = n_emp
        return comp_data_to_store
    except:
          return {'data': 'Not Found'}

In [4]:
import pickle
with open('wiki_dict.pkl', 'rb') as f:
    x = pickle.load(f)
wiki_data_populated = {}
count = 0
for k, v in x.items():
    wiki_data_populated[k] = populate_data(k)
    wiki_data_file = open("wiki_data_populated.pkl", "wb")
    pickle.dump(wiki_data_populated, wiki_data_file)
    wiki_data_file.close()
    count+=1
    if count%100==0:
        print(count)
count

en.wikipedia.org (parse) (24)7.ai
(24)7.ai (en) data
{
  infobox: <dict(11)> name, logo, type, industry, former_name, fou...
  pageid: 14377147
  parsetree: <str(14393)> <root><template><title>short description...
  requests: <list(1)> parse
  title: (24)7.ai
  wikibase: Q4631823
  wikidata_url: https://www.wikidata.org/wiki/Q4631823
  wikitext: <str(11152)> {{short description|Customer experience s...
}
en.wikipedia.org (parse) 280 North, Inc.
280 North, Inc. (en) data
{
  infobox: <dict(5)> name, industry, founded, founder, hq_location...
  pageid: 22320419
  parsetree: <str(4769)> <root><template><title>Infobox company</t...
  requests: <list(1)> parse
  title: 280 North, Inc.
  wikibase: Q4632676
  wikidata_url: https://www.wikidata.org/wiki/Q4632676
  wikitext: <str(3703)> {{Infobox company| name = 280 North| indus...
}
en.wikipedia.org (parse) 2d3
2d3 (en) data
{
  infobox: <dict(5)> name, industry, founded, hq_location, services
  pageid: 42816740
  parsetree: <str(4875)> <root>

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


2251

In [5]:
import pickle
with open('wiki_data_populated.pkl', 'rb') as f:
    x = pickle.load(f)
x
# a_file = open("wiki_data_populated.pkl", "wb")
# pickle.dump(wiki_data_populated, a_file)
# a_file.close()

{'(24)7.ai': {'name': '[24]7.ai, Inc.',
  'type': 'Private',
  'industry': {'Software and Services'},
  'revenue': -1,
  'hq_location': 'San Jose, California',
  'num_employees': 15000},
 '280 North, Inc.': {'name': '280 North',
  'type': '',
  'industry': {'web software'},
  'revenue': -1,
  'hq_location': '',
  'num_employees': 0},
 '2d3': {'name': '2d3 Sensing',
  'type': '',
  'industry': {'software'},
  'revenue': -1,
  'hq_location': 'Irvine, California',
  'num_employees': 0},
 '33Across': {'name': '33Across',
  'type': '',
  'industry': set(),
  'revenue': -1,
  'hq_location': '',
  'num_employees': 0},
 '3D Systems': {'name': '3D Systems Corporation',
  'type': 'Public',
  'industry': {'Technology'},
  'revenue': '646.84 million',
  'hq_location': '',
  'num_employees': 2666},
 '3ality Technica': {'name': '3ality Technica',
  'type': 'Private',
  'industry': {'Entertainment technology'},
  'revenue': -1,
  'hq_location': '',
  'num_employees': 0},
 '4D Inc': {'name': '4D Inc',